In [ ]:
# Install required libraries
!pip install diffusers transformers datasets accelerate torch torchvision --quiet

import torch
from diffusers import StableDiffusionPipeline
from datasets import load_dataset
from torchvision import transforms
import matplotlib.pyplot as plt

# Use a lightweight stable diffusion model instead of DDPM
model_id = "stabilityai/stable-diffusion-2-1-base"

# Load the diffusion model
pipeline = StableDiffusionPipeline.from_pretrained(model_id)
pipeline.to("cuda" if torch.cuda.is_available() else "cpu")

# Load a smaller dataset for efficiency (butterfly images)
dataset = load_dataset("huggan/smithsonian_butterflies_subset", split="train")

# Preprocess dataset (resize & normalize)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

def preprocess(example):
    example["image"] = transform(example["image"])
    return example

dataset = dataset.map(preprocess)

# Function to display dataset images
def show_images():
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for i, img in enumerate(dataset[:5]["image"]):
        img = img.permute(1, 2, 0).numpy()
        axes[i].imshow(img)
        axes[i].axis("off")
    plt.show()

# Show some sample images from dataset
show_images()

# Generate images using a lightweight diffusion model
num_images = 5  # Number of images to generate
prompt = "A beautiful butterfly in a natural habitat"  # Prompt for generation

generated_images = pipeline(prompt, num_inference_steps=25, num_images_per_prompt=num_images).images

# Display generated images
fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
for i, img in enumerate(generated_images):
    axes[i].imshow(img)
    axes[i].axis("off")
plt.show()
